In [1]:
import pandas as pd
import numpy as np

In [2]:
cq_results = pd.read_csv("stat_by_day.csv",sep=';',decimal=',')
cq_results['Name'] = cq_results['DATE'].apply(lambda x: int(''.join(x.split('-'))))
cq_results.sort_values(by="Name",ascending=True, inplace=True)
cq_results.drop(columns=['DATE'],inplace=True)
cq_results.head()

,WEEK,t1,t2,d1,d2,QUANTITY,DRIVER_NBR,Name
0,40,19240.540000,18898.000000,18105.1034,17745.7043,3078.0,116,20201001
1,40,17880.638333,17813.803333,16603.5434,16472.0733,2951.5,120,20201002
2,40,3826.693333,3826.693333,3963.5749,3963.5749,651.5,32,20201003
3,41,19730.351667,19747.641667,18297.9283,18332.5868,3190.0,108,20201005
4,41,23589.926667,23558.828333,22024.4768,22005.9834,3749.5,124,20201006


In [21]:
my_results = pd.read_csv("results.csv",sep=';')
my_results.sort_values(by="Name",ascending=True, inplace=True)
unsolved = list(dict.fromkeys(my_results[my_results['undeliveredCost']>0]['Name']))
my_results = my_results[my_results['undeliveredCost']<1]
print(len(unsolved),' ',  unsolved)

0   []


In [22]:

table = pd.pivot_table(my_results, values=['distanceCost','runtime','satisfiedCost','driverUsed'], index=['Name'],
                           aggfunc={'satisfiedCost': np.max,'distanceCost': np.min,'driverUsed':np.max,
                                    'runtime': np.mean})
table.reset_index(inplace=True)
df = pd.merge(left=table,right=cq_results,how='right',on='Name')
df = df[['Name','satisfiedCost','QUANTITY','distanceCost','d1','d2','DRIVER_NBR','driverUsed','runtime']]
df['LOAD_GAP']= df.apply(lambda x: float(x.satisfiedCost) - float(x.QUANTITY),axis=1)
df['d1_GAP']= df.apply(lambda x: 100*(float(x.d1) - float(x.distanceCost))/float(x.distanceCost),axis=1)
df['d2_GAP']= df.apply(lambda x: 100*(float(x.d2) - float(x.distanceCost))/float(x.distanceCost),axis=1)
df.sort_values(by="d1_GAP",ascending=True, inplace=True)
df

,Name,satisfiedCost,QUANTITY,distanceCost,d1,d2,DRIVER_NBR,driverUsed,runtime,LOAD_GAP,d1_GAP,d2_GAP
5,20201007,2512.5,2512.5,13356.600,12895.9288,12911.9698,124,125.0,971615.0,0.0,-3.449015,-3.328918
18,20201023,3837.5,3837.5,21338.900,20907.9158,20626.2858,131,145.0,1222691.0,0.0,-2.019711,-3.339508
22,20201028,3089.5,3089.5,20859.400,20494.5739,20622.1220,135,149.0,3798482.0,0.0,-1.748977,-1.137511
28,20201104,3953.5,3953.5,22422.300,22082.5415,21978.0572,128,153.0,1553370.0,0.0,-1.515271,-1.981254
14,20201019,3009.5,3009.5,18779.500,18578.1254,18375.3359,121,135.0,554893.0,0.0,-1.072311,-2.152156
39,20201117,3288.5,3288.5,20506.700,20411.5676,20302.4585,124,142.0,3190967.0,0.0,-0.463909,-0.995974
21,20201027,3592.0,3592.0,21113.500,21068.9425,20935.0778,132,158.0,1063819.0,0.0,-0.211038,-0.845062
10,20201014,3471.0,3471.0,20037.700,20080.8966,19876.4187,123,142.0,730914.0,0.0,0.215577,-0.804889
6,20201008,4302.5,4302.5,21729.200,21820.5989,21786.0848,123,143.0,668641.0,0.0,0.420627,0.261790
17,20201022,3428.5,3428.5,23058.800,23215.5066,23143.6320,136,151.0,377085.0,0.0,0.679596,0.367894


In [5]:

table = pd.pivot_table(my_results, values=['travelCost','runtime','satisfiedCost','driverUsed'], index=['Name'],
                           aggfunc={'satisfiedCost': np.max,'travelCost': np.max,'driverUsed':np.max,
                                    'runtime': np.mean})
table.reset_index(inplace=True)
df = pd.merge(left=table,right=cq_results,how='right',on='Name')
df = df[['Name','satisfiedCost','QUANTITY','t1','t2','travelCost','DRIVER_NBR','driverUsed','runtime']]
df['LOAD_GAP']= df.apply(lambda x: float(x.satisfiedCost) - float(x.QUANTITY),axis=1)
df['t1_GAP']= df.apply(lambda x: 100*(float(x.t1) - float(x.travelCost))/float(x.travelCost),axis=1)
df['t2_GAP']= df.apply(lambda x: 100*(float(x.t2) - float(x.travelCost))/float(x.travelCost),axis=1)

df

,Name,satisfiedCost,QUANTITY,t1,t2,travelCost,DRIVER_NBR,driverUsed,runtime,LOAD_GAP,t1_GAP,t2_GAP
0,20201001,3078.0,3078.0,19240.540000,18898.000000,18882.100,116,136.0,45114.000,0.0,1.898306,0.084207
1,20201002,2951.5,2951.5,17880.638333,17813.803333,17718.600,120,133.0,39974.500,0.0,0.914510,0.537307
2,20201003,651.5,651.5,3826.693333,3826.693333,3973.910,32,35.0,2794.000,0.0,-3.704580,-3.704580
3,20201005,3190.0,3190.0,19730.351667,19747.641667,20540.300,108,120.0,36936.500,0.0,-3.943216,-3.859040
4,20201006,3749.5,3749.5,23589.926667,23558.828333,24050.900,124,141.0,58719.000,0.0,-1.916657,-2.045959
5,20201007,2512.5,2512.5,14276.185000,14320.821667,15496.400,124,128.0,31640.500,0.0,-7.874184,-7.586138
6,20201008,NaN,4302.5,23854.651667,23791.373333,NaN,123,NaN,NaN,NaN,NaN,NaN
7,20201009,3229.0,3229.0,17923.901667,17865.196667,18372.600,113,133.0,45297.500,0.0,-2.442215,-2.761739
8,20201010,267.0,267.0,780.841667,780.841667,745.097,13,13.0,1639.000,0.0,4.797317,4.797317
9,20201013,1565.0,1565.0,8453.735000,8417.803333,8775.580,99,104.0,15353.000,0.0,-3.667507,-4.076957
